# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-24-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-24-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-02-25 05:24:57,33.93911,67.709953,55664,2436,49086,4142.0,Afghanistan,142.990938,4.376258
1,NaN,NaN,NaN,Albania,2021-02-25 05:24:57,41.15330,20.168300,103327,1715,66309,35303.0,Albania,3590.485788,1.659779
2,NaN,NaN,NaN,Algeria,2021-02-25 05:24:57,28.03390,1.659600,112461,2970,77537,31954.0,Algeria,256.461403,2.640916
3,NaN,NaN,NaN,Andorra,2021-02-25 05:24:57,42.50630,1.521800,10775,110,10319,346.0,Andorra,13945.512198,1.020882
4,NaN,NaN,NaN,Angola,2021-02-25 05:24:57,-11.20270,17.873900,20640,501,19221,918.0,Angola,62.799950,2.427326


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55514,55518,55540,55557,55575,55580,55604,55617,55646,55664
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,93850,94651,95726,96838,97909,99062,100246,101285,102306,103327
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,110894,111069,111247,111418,111600,111764,111917,112094,112279,112461


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2427,2428,2428,2430,2430,2430,2432,2433,2435,2436
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1567,1582,1600,1617,1636,1653,1666,1681,1696,1715
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2943,2945,2947,2950,2954,2958,2961,2964,2967,2970


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48433,48514,48626,48798,48803,48820,48834,48895,48967,49086
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,57736,58794,59684,60675,61605,62533,63329,64318,65403,66309
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,76168,76330,76489,76640,76797,76940,77076,77225,77382,77537


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6024,6038,6050,6071,6079,6092,6117,6121,6143,6172
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19136,19176,19267,19324,19361,19392,19433,19461,19554,19635
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2044,2055,2053,2057,2061,2067,2070,2074,2084,2095


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,84,84,84,84,85,85,85,85,84,85
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,252,253,253,254,262,262,262,262,263,272
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,48,48,49,49,49,50,50,50,50,50


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-02-25 05:24:57,32.539527,-86.644082,6172,85,0,6087.0,"Autauga, Alabama, US",11047.271295,1.377187
689,1075.0,Lamar,Alabama,US,2021-02-25 05:24:57,33.779950,-88.096680,1316,32,0,1284.0,"Lamar, Alabama, US",9532.777979,2.431611
690,1077.0,Lauderdale,Alabama,US,2021-02-25 05:24:57,34.901719,-87.656247,8570,210,0,8360.0,"Lauderdale, Alabama, US",9241.984708,2.450408


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,28334981,505808,0
India,11046914,156705,10738501
Brazil,10324463,249957,9214337


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,28334981,505808,0,27829173,2021-02-25 05:24:57,37.936303,-91.379001
India,11046914,156705,10738501,151708,2021-02-25 05:24:57,23.088275,81.806127
Brazil,10324463,249957,9214337,860169,2021-02-25 05:24:57,-12.669522,-48.480493
United Kingdom,4156703,121979,11470,4023254,2021-02-25 05:24:57,30.744028,-35.367255
Russia,4153735,83044,3709938,360753,2021-02-25 05:24:57,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3547974,50890,0
Texas,2624852,42831,0
Florida,1885661,30340,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3547974,50890,0,3497084,2021-02-25 05:24:57,37.843962,-120.728594
Texas,2624852,42831,0,2582021,2021-02-25 05:24:57,31.660643,-98.653069
Florida,1885661,30340,0,1855321,2021-02-25 05:24:57,28.940755,-82.700744
New York,1611031,47156,0,1563875,2021-02-25 05:24:57,42.544151,-75.474183
Illinois,1179250,22575,0,1156675,2021-02-25 05:24:57,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1185457,20987,0
Arizona,Maricopa,507480,8938,0
Illinois,Cook,471685,9307,0
Florida,Miami-Dade,405563,5353,0
Texas,Harris,344102,4933,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1185457,20987,0,1164470,2021-02-25 05:24:57,34.308284,-118.228241,6037.0
Arizona,Maricopa,507480,8938,0,498542,2021-02-25 05:24:57,33.348359,-112.491815,4013.0
Illinois,Cook,471685,9307,0,462378,2021-02-25 05:24:57,41.841448,-87.816588,17031.0
Florida,Miami-Dade,405563,5353,0,400210,2021-02-25 05:24:57,25.611236,-80.551706,12086.0
Texas,Harris,344102,4933,0,339169,2021-02-25 05:24:57,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,55617,101285,112094,10712,20548,614,2069751,170506,28937,446644,...,4138237,53310,79681,1,136545,2392,174969,2176,75027,35862
2021-02-23,55646,102306,112279,10739,20584,636,2077228,170672,28939,448371,...,4146760,53973,79717,1,136986,2403,176377,2187,75582,35910
2021-02-24,55664,103327,112461,10775,20640,646,2085411,170945,28947,450376,...,4156702,54874,79749,1,137445,2412,177768,2221,76484,35960


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,2433,1681,2964,109,499,14,51359,3167,909,8397,...,120988,583,622,0,1320,35,1986,619,1031,1441
2021-02-23,2435,1696,2967,110,500,14,51510,3171,909,8434,...,121536,588,622,0,1325,35,1994,620,1040,1448
2021-02-24,2436,1715,2970,110,501,14,51650,3175,909,8470,...,121978,591,622,0,1331,35,1999,624,1051,1456


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,48895,64318,77225,10245,19190,222,1872213,162098,22885,421811,...,11346,46845,78171,1,128579,1717,162025,1432,68928,32216
2021-02-23,48967,65403,77382,10285,19207,233,1877898,162329,22892,423014,...,11457,47512,78185,1,129027,1760,162757,1432,69436,32288
2021-02-24,49086,66309,77537,10319,19221,233,1882568,162517,22897,424348,...,11469,48035,78252,1,129487,1804,163795,1433,69803,32410


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6024,6038,6050,6071,6079,6092,6117,6121,6143,6172
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1281,1283,1282,1286,1291,1292,1294,1296,1306,1316
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8476,8479,8480,8508,8513,8515,8523,8525,8541,8570
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2780,2781,2781,2787,2790,2794,2795,2798,2806,2812
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,14622,14641,14684,14729,14766,14779,14798,14809,14827,14883


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-22,487520,57738,0,809474,315759,3536946,421294,275334,85090,49,...,766089,2606513,367073,14608,2579,565270,334962,129854,612712,53900
2021-02-23,488973,57793,0,810658,316593,3543340,422390,276691,85228,49,...,767315,2617096,367789,14691,2589,567039,335693,130139,613419,53944
2021-02-24,490220,57970,0,811968,317396,3547974,423558,278184,85506,49,...,768946,2624852,368601,14768,2623,568946,336565,130382,614260,53988


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-02-22        6121   19461    2074  2417   6043    1153   1945   13017   
2021-02-23        6143   19554    2084  2432   6058    1160   1948   13063   
2021-02-24        6172   19635    2095  2437   6072    1165   1951   13090   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-22         3367     1753  ...    2603    583     2997      671   
2021-02-23         3382     1757  ...    2603    583     2999      671   
2021-02-24         3393     1763  ...    2604    585     3002      672   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-22           3686  3331  2030          0      879    620  
2021-02-23           3699  3333  2036          0      880    621  
2021-02-24           3702  3336  2037          0      880    622  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-22,9592,290,0,15502,5363,49541,5893,7562,1368,0,...,11153,42301,1853,198,25,7486,4857,2263,6871,662
2021-02-23,9660,290,0,15650,5377,49888,5907,7572,1379,0,...,11198,42521,1865,199,25,7658,4881,2274,6910,671
2021-02-24,9744,290,0,15693,5387,50890,5917,7595,1402,0,...,11266,42831,1879,201,25,7807,4912,2285,6936,671


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-02-22          85     262      50   58    125      33     64     278   
2021-02-23          84     263      50   59    125      33     65     281   
2021-02-24          85     272      50   59    125      34     65     283   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-22           99       37  ...      26     11       27        7   
2021-02-23          102       37  ...      26     11       27        7   
2021-02-24          103       38  ...      26     11       27        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-22             34     9    12          0       26      5  
2021-02-23             34     9    12          0       26      5  
2021-02-24             34     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,0.000234,0.010365,0.001582,0.001215,0.001413,0.026756,0.002624,0.000610,0.000242,0.002852,...,0.002582,0.009372,0.000339,0.0,0.003506,0.003777,0.007683,0.005081,0.007033,0.001844
2021-02-23,0.000521,0.010080,0.001650,0.002521,0.001752,0.035831,0.003613,0.000974,0.000069,0.003867,...,0.002060,0.012437,0.000452,0.0,0.003230,0.004599,0.008047,0.005055,0.007397,0.001338
2021-02-24,0.000323,0.009980,0.001621,0.003352,0.002721,0.015723,0.003939,0.001600,0.000276,0.004472,...,0.002398,0.016694,0.000401,0.0,0.003351,0.003745,0.007887,0.015546,0.011934,0.001392


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,0.000411,0.009004,0.001013,0.018692,0.000000,0.076923,0.003145,0.000948,0.0,0.001312,...,0.001473,0.015679,0.0,NaN,0.003040,0.0,0.005061,0.000000,0.010784,0.003482
2021-02-23,0.000822,0.008923,0.001012,0.009174,0.002004,0.000000,0.002940,0.001263,0.0,0.004406,...,0.004529,0.008576,0.0,NaN,0.003788,0.0,0.004028,0.001616,0.008729,0.004858
2021-02-24,0.000411,0.011203,0.001011,0.000000,0.002000,0.000000,0.002718,0.001261,0.0,0.004268,...,0.003637,0.005102,0.0,NaN,0.004528,0.0,0.002508,0.006452,0.010577,0.005525


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,0.001249,0.015617,0.001933,0.003821,0.009309,0.018349,0.003060,0.000642,0.000131,0.003096,...,0.002740,0.012536,0.000243,0.0,0.003833,0.008813,0.003810,0.000000,0.014483,0.002833
2021-02-23,0.001473,0.016869,0.002033,0.003904,0.000886,0.049550,0.003037,0.001425,0.000306,0.002852,...,0.009783,0.014238,0.000179,0.0,0.003484,0.025044,0.004518,0.000000,0.007370,0.002235
2021-02-24,0.002430,0.013853,0.002003,0.003306,0.000729,0.000000,0.002487,0.001158,0.000218,0.003154,...,0.001047,0.011008,0.000857,0.0,0.003565,0.025000,0.006378,0.000698,0.005285,0.003778


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-22,0.001391,0.006011,NaN,0.001865,0.000777,0.001444,0.001617,0.008176,0.004225,0.0,...,0.001244,0.002251,0.000922,0.007935,0.001553,0.002047,0.003499,0.001836,0.000771,0.001952
2021-02-23,0.002980,0.000953,NaN,0.001463,0.002641,0.001808,0.002602,0.004929,0.001622,0.0,...,0.001600,0.004060,0.001951,0.005682,0.003877,0.003129,0.002182,0.002195,0.001154,0.000816
2021-02-24,0.002550,0.003063,NaN,0.001616,0.002536,0.001308,0.002765,0.005396,0.003262,0.0,...,0.002126,0.002964,0.002208,0.005241,0.013132,0.003363,0.002598,0.001867,0.001371,0.000816


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-22      0.000654  0.001441  0.001932  0.000414  0.000166  0.001738   
2021-02-23      0.003594  0.004779  0.004822  0.006206  0.002482  0.006071   
2021-02-24      0.004721  0.004142  0.005278  0.002056  0.002311  0.004310   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-22      0.002577  0.005951  0.000000  0.000000  ...  0.002310   
2021-02-23      0.001542  0.003534  0.004455  0.002282  ...  0.000000   
2021-02-24      0.001540  0.002067  0.003253  0.003415  ...  0.000384   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-02-22      0.005172  0.001671  0.00000   0.007104  0.001503  0.001480   
2021-02-23      0.000000  0.000667  0.00000   0.003527  0.000600  0.002956   
2021-02-24      0.003431  0.001000  0.00149   0.000811  0.000900  0.000491   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-22            NaN  0.002281  0.001616  
2021-02-23            NaN  0.001138  0.001613  
2021-02-24            NaN  0.000000  0.001610  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-22,0.000000,0.00346,NaN,-0.000193,0.00112,0.003972,0.000170,0.005184,0.000732,NaN,...,0.001796,0.003297,0.000540,0.005076,0.0,0.021143,0.007258,0.000885,0.000000,0.000000
2021-02-23,0.007089,0.00000,NaN,0.009547,0.00261,0.007004,0.002376,0.001322,0.008041,NaN,...,0.004035,0.005201,0.006476,0.005051,0.0,0.022976,0.004941,0.004861,0.005676,0.013595
2021-02-24,0.008696,0.00000,NaN,0.002748,0.00186,0.020085,0.001693,0.003038,0.016679,NaN,...,0.006073,0.007291,0.007507,0.010050,0.0,0.019457,0.006351,0.004837,0.003763,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                               \
Admin2           Autauga   Baldwin Barbour      Bibb Blount   Bullock   
2021-02-22      0.000000  0.000000     0.0  0.000000    0.0  0.000000   
2021-02-23     -0.011765  0.003817     0.0  0.017241    0.0  0.000000   
2021-02-24      0.011905  0.034221     0.0  0.000000    0.0  0.030303   

Province_State                                          ... Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2021-02-22      0.000000  0.000000  0.000000  0.000000  ...     0.0    0.0   
2021-02-23      0.015625  0.010791  0.030303  0.000000  ...     0.0    0.0   
2021-02-24      0.000000  0.007117  0.009804  0.027027  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2021-02-22          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-02-23          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-02-24          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2021-02-22        0.0  
2021-02-23        0.0  
2021-02-24        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,0.000276,0.011021,0.001518,0.001924,0.001259,0.029296,0.002561,0.000858,0.000196,0.003503,...,0.002575,0.011119,0.000363,0.0,0.003415,0.004831,0.006693,0.003640,0.008089,0.001803
2021-02-23,0.000399,0.010551,0.001584,0.002222,0.001506,0.032563,0.003087,0.000916,0.000132,0.003685,...,0.002317,0.011778,0.000408,0.0,0.003322,0.004715,0.007370,0.004348,0.007743,0.001571
2021-02-24,0.000361,0.010265,0.001603,0.002787,0.002113,0.024143,0.003513,0.001258,0.000204,0.004078,...,0.002357,0.014236,0.000405,0.0,0.003336,0.004230,0.007628,0.009947,0.009839,0.001481


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,0.000443,0.009148,0.001072,0.009355,0.000727,0.098176,0.002656,0.001001,3.820977e-21,0.001956,...,0.002223,0.012557,3.931419e-07,NaN,0.003189,3.321769e-54,0.004181,0.000421,0.009420,0.003211
2021-02-23,0.000632,0.009036,0.001042,0.009265,0.001365,0.049088,0.002798,0.001132,1.910488e-21,0.003181,...,0.003376,0.010567,1.965710e-07,NaN,0.003488,1.660884e-54,0.004105,0.001018,0.009075,0.004035
2021-02-24,0.000522,0.010119,0.001027,0.004632,0.001683,0.024544,0.002758,0.001197,9.552442e-22,0.003725,...,0.003506,0.007834,9.828548e-08,NaN,0.004008,8.304422e-55,0.003306,0.003735,0.009826,0.004780


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-22,0.000911,0.014852,0.001898,0.003630,0.005008,0.012968,0.003009,0.000680,0.000108,0.003143,...,0.003128,0.010975,0.000316,0.0,0.003709,0.017611,0.003960,0.000028,0.014381,0.003072
2021-02-23,0.001192,0.015861,0.001966,0.003767,0.002947,0.031259,0.003023,0.001052,0.000207,0.002998,...,0.006455,0.012607,0.000248,0.0,0.003597,0.021327,0.004239,0.000014,0.010875,0.002653
2021-02-24,0.001811,0.014857,0.001984,0.003537,0.001838,0.015629,0.002755,0.001105,0.000213,0.003076,...,0.003751,0.011807,0.000552,0.0,0.003581,0.023164,0.005308,0.000356,0.008080,0.003216


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-22,0.001573,0.003443,NaN,0.002057,0.000947,0.001535,0.001976,0.004520,0.003653,1.899913e-104,...,0.001402,0.002300,0.001518,0.008249,0.001404,0.002948,0.002463,0.002004,0.000913,0.001787
2021-02-23,0.002277,0.002198,NaN,0.001760,0.001794,0.001671,0.002289,0.004724,0.002638,9.499563e-105,...,0.001501,0.003180,0.001734,0.006966,0.002641,0.003039,0.002323,0.002099,0.001033,0.001301
2021-02-24,0.002414,0.002630,NaN,0.001688,0.002165,0.001489,0.002527,0.005060,0.002950,4.749781e-105,...,0.001813,0.003072,0.001971,0.006103,0.007887,0.003201,0.002460,0.001983,0.001202,0.001059


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-22      0.001874  0.001769  0.001913  0.000819  0.000808  0.001715   
2021-02-23      0.002734  0.003274  0.003367  0.003512  0.001645  0.003893   
2021-02-24      0.003728  0.003708  0.004323  0.002784  0.001978  0.004102   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-22      0.002184  0.004324  0.000773  0.000190  ...  0.002059   
2021-02-23      0.001863  0.003929  0.002614  0.001236  ...  0.001030   
2021-02-24      0.001702  0.002998  0.002933  0.002325  ...  0.000707   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-22      0.003754  0.001290  0.000103   0.005137  0.001706  0.001402   
2021-02-23      0.001877  0.000979  0.000051   0.004332  0.001153  0.002179   
2021-02-24      0.002654  0.000989  0.000771   0.002571  0.001027  0.001335   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-22           -1.0  0.001329  0.000934  
2021-02-23           -1.0  0.001233  0.001274  
2021-02-24           -1.0  0.000617  0.001442  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-22,0.001709,0.002144,NaN,0.001931,0.001591,0.005052,0.000817,0.002998,0.000604,NaN,...,0.002195,0.003474,0.003118,0.006151,0.000010,0.017228,0.004397,0.002138,0.000834,0.000183
2021-02-23,0.004399,0.001072,NaN,0.005739,0.002101,0.006028,0.001596,0.002160,0.004322,NaN,...,0.003115,0.004337,0.004797,0.005601,0.000005,0.020102,0.004669,0.003500,0.003255,0.006889
2021-02-24,0.006547,0.000536,NaN,0.004243,0.001980,0.013057,0.001645,0.002599,0.010501,NaN,...,0.004594,0.005814,0.006152,0.007826,0.000003,0.019780,0.005510,0.004168,0.003509,0.003445


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-22      0.000772  0.002156  0.002901  0.001126  0.002091  0.000505   
2021-02-23     -0.005496  0.002987  0.001450  0.009184  0.001046  0.000252   
2021-02-24      0.003204  0.018604  0.000725  0.004592  0.000523  0.015278   

Province_State                                              ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...      Park   
2021-02-22      0.000004  0.003478  0.001331  1.710204e-06  ...  0.000318   
2021-02-23      0.007815  0.007135  0.015817  8.551020e-07  ...  0.000159   
2021-02-24      0.003907  0.007126  0.012810  1.351394e-02  ...  0.000079   

Province_State                                                                \
Admin2                Platte  Sheridan      Sublette    Sweetwater     Teton   
2021-02-22      3.757630e-10  0.001362  3.071617e-16  1.864719e-06  0.000981   
2021-02-23      1.878815e-10  0.000681  1.535809e-16  9.323593e-07  0.000490   
2021-02-24      9.394075e-11  0.000340  7.679043e-17  4.661796e-07  0.000245   

Province_State                                                       
Admin2                 Uinta Unassigned      Washakie        Weston  
2021-02-22      4.982932e-11  -0.501961  1.908447e-08  4.053116e-07  
2021-02-23      2.491466e-11  -0.501961  9.542237e-09  2.026558e-07  
2021-02-24      1.245733e-11  -0.501961  4.771119e-09  1.013279e-07  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210224,AK,55736,NaN,NaN,NaN,totalTestsViral,1662156,46.0,1260.0,...,NaN,0,0,67b3b6ca1627ea40d08871803b2659b08b55daae,0,0,0,0,0,NaN
1,20210224,AL,490220,106338.0,1885151.0,NaN,totalTestsPeopleViral,2269033,773.0,45250.0,...,NaN,84,0,676bdea053983e254017ca1a5c4545ebe6b40100,0,0,0,0,0,NaN
2,20210224,AR,317396,66671.0,2367951.0,NaN,totalTestsViral,2618676,545.0,14649.0,...,NaN,10,32,2db69acfbfe82aa932fd048848b370f4d670e601,0,0,0,0,0,NaN
3,20210224,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f43db694c3c66828b057fcd5303d23ff2014fad3,0,0,0,0,0,NaN
4,20210224,AZ,811968,54615.0,2960197.0,NaN,totalTestsViral,7512395,1449.0,57156.0,...,NaN,43,84,66eb7b9f8629ac10b33a2ddb54fadd311466dd44,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210224,AK,55736.0,NaN,NaN,NaN,totalTestsViral,1662156.0,46.0,1260.0,...,NaN,0,0,67b3b6ca1627ea40d08871803b2659b08b55daae,0,0,0,0,0,NaN
1,20210224,AL,490220.0,106338.0,1885151.0,NaN,totalTestsPeopleViral,2269033.0,773.0,45250.0,...,NaN,84,0,676bdea053983e254017ca1a5c4545ebe6b40100,0,0,0,0,0,NaN
2,20210224,AR,317396.0,66671.0,2367951.0,NaN,totalTestsViral,2618676.0,545.0,14649.0,...,NaN,10,32,2db69acfbfe82aa932fd048848b370f4d670e601,0,0,0,0,0,NaN
3,20210224,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f43db694c3c66828b057fcd5303d23ff2014fad3,0,0,0,0,0,NaN
4,20210224,AZ,811968.0,54615.0,2960197.0,NaN,totalTestsViral,7512395.0,1449.0,57156.0,...,NaN,43,84,66eb7b9f8629ac10b33a2ddb54fadd311466dd44,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-24,AK,55736,NaN,NaN,NaN,totalTestsViral,1662156,46.0,1260.0,NaN,...,NaN,0,0,67b3b6ca1627ea40d08871803b2659b08b55daae,0,0,0,0,0,NaN
2021-02-24,AL,490220,106338.0,1885151.0,NaN,totalTestsPeopleViral,2269033,773.0,45250.0,NaN,...,NaN,84,0,676bdea053983e254017ca1a5c4545ebe6b40100,0,0,0,0,0,NaN
2021-02-24,AR,317396,66671.0,2367951.0,NaN,totalTestsViral,2618676,545.0,14649.0,204.0,...,NaN,10,32,2db69acfbfe82aa932fd048848b370f4d670e601,0,0,0,0,0,NaN
2021-02-24,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f43db694c3c66828b057fcd5303d23ff2014fad3,0,0,0,0,0,NaN
2021-02-24,AZ,811968,54615.0,2960197.0,NaN,totalTestsViral,7512395,1449.0,57156.0,430.0,...,NaN,43,84,66eb7b9f8629ac10b33a2ddb54fadd311466dd44,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-24,AK,55736.0,NaN,NaN,NaN,totalTestsViral,1662156.0,46.0,1260.0,NaN,...,NaN,0,0,67b3b6ca1627ea40d08871803b2659b08b55daae,0,0,0,0,0,NaN
2021-02-24,AL,490220.0,106338.0,1885151.0,NaN,totalTestsPeopleViral,2269033.0,773.0,45250.0,NaN,...,NaN,84,0,676bdea053983e254017ca1a5c4545ebe6b40100,0,0,0,0,0,NaN
2021-02-24,AR,317396.0,66671.0,2367951.0,NaN,totalTestsViral,2618676.0,545.0,14649.0,204.0,...,NaN,10,32,2db69acfbfe82aa932fd048848b370f4d670e601,0,0,0,0,0,NaN
2021-02-24,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f43db694c3c66828b057fcd5303d23ff2014fad3,0,0,0,0,0,NaN
2021-02-24,AZ,811968.0,54615.0,2960197.0,NaN,totalTestsViral,7512395.0,1449.0,57156.0,430.0,...,NaN,43,84,66eb7b9f8629ac10b33a2ddb54fadd311466dd44,0,0,0,0,0,NaN
